<a href="https://colab.research.google.com/github/William-Kent/Basic_Web_App/blob/master/Sandbox/Prototype_Falcon_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Falcon 7B
This notebook builds on [this](https://github.com/andrewgcodes/FalconStreaming/blob/main/Falcon7B_Instruct_Streaming.ipynb) notebook created by Andrew Gao.


In [1]:
!pip install git+https://www.github.com/huggingface/transformers
!pip install git+https://github.com/huggingface/accelerate
!pip install bitsandbytes
!pip install einops

from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import torch

  Cloning https://www.github.com/huggingface/transformers to /tmp/pip-req-build-x5b2borq
  Running command git clone --filter=blob:none --quiet https://www.github.com/huggingface/transformers /tmp/pip-req-build-x5b2borq
  Resolved https://www.github.com/huggingface/transformers to commit 0afa5071bd84e44301750fdc594e33db102cf374
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.7 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.33.0.dev0-py3-none-any.whl size=7634320 sha256=66554bd93416fe02a3ca1ef3d8cb08347702cacf727f97d3c476c7b5563d288a
  Stored in directory: /tmp/pip-ephem-wheel-cache-_kdse9ng/wheels/c1/ac/11/e69d454307e735e14f4f95e575c8be27fd99835ec36f504c13


In [ ]:
model_path="tiiuae/falcon-7b-instruct"

config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, load_in_4bit=True, device_map="auto")

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct")

In [ ]:
input_text = "In a table, tell me the sentiment, topic, subtopic, location and the crisis for which this tweet relates: RT @CBCAlerts: Canmore, Alta. declares state of emergency due to flooding  - with some residents being moved to community centre #Alberta"

In [ ]:
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

next_input = input_ids
# Change to required length, too long and may cause OOM issue
max_length = 120
current_length = input_ids.shape[1]

while True:
  # Check if we've reached the length limit
  if current_length >= max_length:
    break

  output = model(next_input)
  next_token_logits = output.logits[:, -1, :]
  next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
  print(tokenizer.decode(next_token_id[0].cpu().tolist(), skip_special_tokens=True), end='', flush=True)

  next_input = torch.cat([next_input, next_token_id.to("cuda")], dim=-1)

  current_length += 1

  if next_token_id[0].item() == tokenizer.eos_token_id:
      break
